In [1]:
import os
import re
import json
import operator
import codecs

from typing import List, Union, Dict
from pprint import pprint
from functools import reduce

import enchant
import pandas as pd
import numpy as np
from pyxdameraulevenshtein import damerau_levenshtein_distance

In [2]:
def read_json(path: str) -> Union[List, Dict]:
    with open(path) as fp:
        return json.load(fp)


In [3]:
%ls zoomtail

Fabfirki.json         Prachi Creation.json


In [4]:
zoom_fabfirki: list = read_json('zoomtail/Fabfirki.json'); type(zoom_fabfirki)

list

In [5]:
zoom_prachi: list = read_json('zoomtail/Prachi Creation.json'); type(zoom_prachi)

list

In [6]:
def get_all_values_of_keys_from_list_of_objs(list_obj: list, keys: list, transpose_on: bool = True) -> np.array:
    return_list = []
    for key in keys:
        try:
            final_key = []
            key = key.split('.')
            for ind_key in key:
                final_key.append(int(ind_key) if len(ind_key) == 1 else ind_key)
            
            individual_values = []
            for obj in list_obj:
                try:
                    reduced_val = reduce(operator.getitem, final_key, obj)
                
                except Exception:
                    reduced_val = np.nan
                individual_values.append(reduced_val)
                
            return_list.append(individual_values)
            
        except KeyError as k:
            print('Incorrect Key Entered', k)
        
    if transpose_on:
        return np.array(return_list).T
    return np.array(return_list)


def get_list_of_dicts_from_json(path: str) -> list:
    """
    Open a json file that has a list of strings(that are actually dicts)
    and returns them as a list of dicts
    """
    
    with open(path) as fp:
        data = json.load(fp)
    return [json.loads(strings) for strings in data]


In [7]:
keys_to_get = ['zsn', 'catalogName', 'sellingPrice', 'brand.0.brandName', 'fabricData.0.fabricName', 'tags',]

In [8]:
df_zoom_fab = pd.DataFrame(get_all_values_of_keys_from_list_of_objs(zoom_fabfirki, keys=keys_to_get),
                           columns=['zsn', 'catalogName', 'sellingPrice', 'brandName', 'fabricName', 'tags'])
df_zoom_prach = pd.DataFrame(get_all_values_of_keys_from_list_of_objs(zoom_prachi, keys=keys_to_get),
                             columns=['zsn', 'catalogName', 'sellingPrice', 'brandName', 'fabricName', 'tags'])

In [9]:
df_zoom_fab.head()

,zsn,catalogName,sellingPrice,brandName,fabricName,tags
0,ZT-KU-2016,Cadbury,760,Fabfirki,Khadi Cotton,[]
1,ZT-KU-2015,Madhubala,815,Fabfirki,Flex Rayon,[]
2,ZT-KU-2023,Rose Rivera,480,Fabfirki,14kg Rayon,[]
3,201-001141,Anaya 1,474,Multi Brand,14kg Rayon,[]
4,201-001128,Ami..Sa,540,Multi Brand,Slub Rayon,[]


In [10]:
df_zoom_prach.tags[0]

[{'attribute': 'Pattern', 'value': 'Printed'},
 {'attribute': 'Topwear style', 'value': 'Layered'},
 {'attribute': 'Neck', 'value': 'Round'},
 {'attribute': 'Topwear length', 'value': 'Ankle'},
 {'attribute': 'Colour Palette', 'value': 'Dark colours'},
 {'attribute': 'Sleeve style', 'value': 'Sleeveless'}]

In [11]:
base_dir = 'prachi-collection/'
prachi_collection_list = []

for ind_listings in os.listdir(base_dir):
    prachi_collection_list.extend(get_list_of_dicts_from_json(base_dir + ind_listings))

In [12]:
udaan_keys_to_get = ['listingId', 'title', "description", "specList", "minPricePaise"]

In [18]:
df_udaan_prach = pd.DataFrame(get_all_values_of_keys_from_list_of_objs(prachi_collection_list, keys=udaan_keys_to_get),
                             columns=udaan_keys_to_get)

df_udaan_prach['minPrice'] = df_udaan_prach['minPricePaise'] / 100
df_udaan_prach.drop(columns='minPricePaise', axis=1, inplace=True)

In [19]:
df_udaan_prach.head()

,listingId,title,description,specList,minPrice
0,TLCKR67E2Q0S672DYYFRJ81ZZ47GTCE,Slub COTTON KURTIS. 6 pieces set,<p>Call / whats app 7096358008</p>\n,"[{'fieldName': 'ctg_fabric', 'fieldDisplayName...",205
1,TLCKR2XXWJR6DN08514NV4K707TTESL,RAYON LONG DESIGNER DOUBLE LAYER KURTIS. 4 CO...,<p>Call / whats app 7096358008</p>\n,"[{'fieldName': 'ctg_ideal_for', 'fieldDisplayN...",NaN
2,TLCKR61D16QYCQCD2945VKBBYYPQ5F9,D. No. 1217. 14kg RAYON Embroidered KURTIS. 4 ...,<p>Call / whats app 7096358008</p>\n,"[{'fieldName': 'ctg_fabric', 'fieldDisplayName...",203.7
3,TLCKR10JHREK9P086LZ748DGT4HS9RK,Cotton GHERA pattern KURTIS. 4 COLOUR SET,<p>Call / whats app 7096358008</p>\n,"[{'fieldName': 'ctg_fabric', 'fieldDisplayName...",165
4,TLCKRBY8TQQECENCT9ZZF8408D2BRET,Rayon long designer double layer kurtis. 4 col...,<p>Call/whats app 7096358008</p>\n,"[{'fieldName': 'ctg_ideal_for', 'fieldDisplayN...",240


In [20]:
base_dir = 'fabfirki/'
fab_collection_list = []

for ind_listings in os.listdir(base_dir):
    fab_collection_list.extend(get_list_of_dicts_from_json(base_dir + ind_listings))

len(fab_collection_list)

93

In [21]:
df_udaan_fab = pd.DataFrame(get_all_values_of_keys_from_list_of_objs(fab_collection_list, keys=udaan_keys_to_get),
                             columns=udaan_keys_to_get)

df_udaan_fab['minPrice'] = df_udaan_fab['minPricePaise'] / 100
df_udaan_fab.drop(columns='minPricePaise', axis=1, inplace=True)
df_udaan_fab.head()

,listingId,title,description,specList,minPrice
0,TLCKR48P43RQTV084EZJZT8N3LY3N96,Fabfirki New Letest gown kurti size set,"<p><em>Fabrics</em> :- 14kg Reyon, Cottan,cora...","[{'fieldName': 'size', 'fieldDisplayName': 'Si...",NaN
1,TLCKR1MMN8C9B4Z8HRFMJGXDS3E0EDF,Fabfirki New Letest designer catlog Mahi,<p>Catalogue: - <em>MAHI</em>\nFabrics: - He...,"[{'fieldName': 'ctg_ideal_for', 'fieldDisplayN...",NaN
2,TLCKRX2HMX7CEJJCGE4WRE5TPWVQR9C,Fabfirki New Letest Catlog Sanskruti,<p>FASHION SHEELA VOLUME 01</p>\n<p>Fabric: SA...,"[{'fieldName': 'ctg_ideal_for', 'fieldDisplayN...",NaN
3,TLCKR1JTYGW57GC8CEZ4SCKNZNXQKH0,Fabfirki Rayon print anarkali kurti catlog Blu...,<p>Cataloguen Name :-<em>Blue Whale</em>\nClot...,"[{'fieldName': 'ctg_ideal_for', 'fieldDisplayN...",NaN
4,TLCKRL97CKLS0C58LWZHPMW6ZRPNJYZ,Fabfirki Rayon Fancy embroidery kurti catlog P...,<p>Catlog Name: <em>P A I S L E Y</em>\nA New...,"[{'fieldName': 'size', 'fieldDisplayName': 'Si...",NaN


In [22]:
def apply_regex(pandas_series: pd.Series, regex_to_apply: Union[str, re.Pattern],
                          replace_with: str = '') -> pd.Series:
    """
    This function applies the given regex to the series and returns it
    """
    return pandas_series.apply(lambda x: re.sub(regex_to_apply, replace_with, x))


def remove_newline_character(obj: Union[str, pd.Series]) -> Union[str, pd.Series]:
    """
    Removes newline character either from the series or str, whichever is specified
    """
    if isinstance(obj, str):
        return obj.replace('\n', '')
    return obj.apply(lambda x: x.replace('\n', ''))

def replace_none_by_nan(df: pd.DataFrame, inplace: bool = False) -> Union[pd.DataFrame, None]:
        return df.fillna(np.nan, inplace=inplace)

def get_unique_counts(series_obj: pd.Series) -> str:
    return f"Original count is: {len(series_obj)} and unique count is: {len(np.unique(series_obj))}"

def drop_duplicates(df: pd.DataFrame, column_to_match: str) -> None:
    """
    This method drops the duplicate values from a df based on column_to_match
    """
    original_len = len(df[column_to_match])
    df.drop_duplicates(subset=column_to_match, inplace=True)
    print(f"Dropped {original_len - len(df[column_to_match])} non-unique rows")

In [23]:
# Write a func to modify the description column
regex_to_remove = re.compile(r'(<\w+>|<\/\w+>|Â\xa0)')

df_udaan_fab['description'] = remove_newline_character(apply_regex(df_udaan_fab.description, regex_to_remove))
df_udaan_fab.head()

,listingId,title,description,specList,minPrice
0,TLCKR48P43RQTV084EZJZT8N3LY3N96,Fabfirki New Letest gown kurti size set,"Fabrics :- 14kg Reyon, Cottan,cora silk,Embroi...","[{'fieldName': 'size', 'fieldDisplayName': 'Si...",NaN
1,TLCKR1MMN8C9B4Z8HRFMJGXDS3E0EDF,Fabfirki New Letest designer catlog Mahi,Catalogue: - MAHIFabrics: - Heavy 14kg Plain...,"[{'fieldName': 'ctg_ideal_for', 'fieldDisplayN...",NaN
2,TLCKRX2HMX7CEJJCGE4WRE5TPWVQR9C,Fabfirki New Letest Catlog Sanskruti,FASHION SHEELA VOLUME 01Fabric: SANSKRUTIAvail...,"[{'fieldName': 'ctg_ideal_for', 'fieldDisplayN...",NaN
3,TLCKR1JTYGW57GC8CEZ4SCKNZNXQKH0,Fabfirki Rayon print anarkali kurti catlog Blu...,Cataloguen Name :-Blue WhaleCloth :- Rayon Bla...,"[{'fieldName': 'ctg_ideal_for', 'fieldDisplayN...",NaN
4,TLCKRL97CKLS0C58LWZHPMW6ZRPNJYZ,Fabfirki Rayon Fancy embroidery kurti catlog P...,Catlog Name: P A I S L E YA New Range of Rayo...,"[{'fieldName': 'size', 'fieldDisplayName': 'Si...",NaN


In [24]:
df_udaan_prach['description'] = remove_newline_character(apply_regex(replace_none_by_nan(df_udaan_prach)
                                                                     ['description'].fillna(''), regex_to_remove))
df_udaan_prach.head()

,listingId,title,description,specList,minPrice
0,TLCKR67E2Q0S672DYYFRJ81ZZ47GTCE,Slub COTTON KURTIS. 6 pieces set,Call / whats app 7096358008,"[{'fieldName': 'ctg_fabric', 'fieldDisplayName...",205
1,TLCKR2XXWJR6DN08514NV4K707TTESL,RAYON LONG DESIGNER DOUBLE LAYER KURTIS. 4 CO...,Call / whats app 7096358008,"[{'fieldName': 'ctg_ideal_for', 'fieldDisplayN...",NaN
2,TLCKR61D16QYCQCD2945VKBBYYPQ5F9,D. No. 1217. 14kg RAYON Embroidered KURTIS. 4 ...,Call / whats app 7096358008,"[{'fieldName': 'ctg_fabric', 'fieldDisplayName...",203.7
3,TLCKR10JHREK9P086LZ748DGT4HS9RK,Cotton GHERA pattern KURTIS. 4 COLOUR SET,Call / whats app 7096358008,"[{'fieldName': 'ctg_fabric', 'fieldDisplayName...",165
4,TLCKRBY8TQQECENCT9ZZF8408D2BRET,Rayon long designer double layer kurtis. 4 col...,Call/whats app 7096358008,"[{'fieldName': 'ctg_ideal_for', 'fieldDisplayN...",240


In [25]:
# drop all duplicates from data
drop_duplicates(df_zoom_fab, 'zsn')
drop_duplicates(df_zoom_prach, 'zsn')
drop_duplicates(df_udaan_fab, 'listingId')
drop_duplicates(df_udaan_prach, 'listingId')

Dropped 0 non-unique rows
Dropped 0 non-unique rows
Dropped 10 non-unique rows
Dropped 201 non-unique rows


In [26]:
# convert nan in title and description columns to ''
df_udaan_fab['title'].fillna('', inplace=True)
df_udaan_fab['description'].fillna('', inplace=True)
df_udaan_prach['title'].fillna('', inplace=True)
df_udaan_prach['description'].fillna('', inplace=True)

In [69]:
(df_zoom_fab['catalogName'] + ' ' + df_zoom_fab['brandName'] + ' '+  df_zoom_fab['fabricName'])[:5]

0      Cadbury Fabfirki Khadi Cotton
1      Madhubala Fabfirki Flex Rayon
2    Rose Rivera Fabfirki 14kg Rayon
3     Anaya 1 Multi Brand 14kg Rayon
4     Ami..Sa Multi Brand Slub Rayon
dtype: object

In [56]:
print((df_udaan_fab['title'] + df_udaan_fab['description'])[0], end = '\n\n')
print((df_udaan_fab['title'] + df_udaan_fab['description'])[1], end = '\n\n')
print((df_udaan_fab['title'] + df_udaan_fab['description'])[2], end = '\n\n')

fabfirki new leftest gown kurtis size setfabrics 14kg rayon cotton cora silk embroidery work hand block print full interlock bombay finishing size m to xxllength gt 50th 52 rate 800 5 st

fabfirki new leftest designer catalog magicatalogue mahifabrics heavy 14kg plain rayon pure cotton check with hand work size xl 42 length 46 quot price 450 âž 5 st

fabfirki new leftest catalog sanskrutifashion sheela volume 01fabric sanskrutiavailable size m l xl xxl 3xllength borate 425cataloge 8 peace showroom finishing full stitched wash care hand and machine wash dispatch date 10 8 2018



In [66]:
'mahifabrics' in df_zoom_fab.catalogName.apply(lambda x: x.lower()).values

False

In [ ]:
# matching algo based on conditionals
df_udaan_fab['combo_for_matching'] = df_udaan_fab['title'] + df_udaan_fab['description']

In [84]:
total_sum = 0
for fabric_name in df_zoom_fab['fabricName'][:5]:
#     total_sum += sum(df_udaan_fab['combo_for_matching'].str.contains(fabric_name.lower()))
    print([val.in for val in df_udaan_fab['combo_for_matching'].str.findall(fabric_name, flags=re.IGNORECASE) if val])
          
# total_sum

[]
[]
[<built-in method index of list object at 0x11d9fee88>, <built-in method index of list object at 0x11d9dd4c8>, <built-in method index of list object at 0x11da09388>, <built-in method index of list object at 0x11da09b08>, <built-in method index of list object at 0x11d9fb988>, <built-in method index of list object at 0x11d9256c8>]
[<built-in method index of list object at 0x11d925ac8>, <built-in method index of list object at 0x11d921c48>, <built-in method index of list object at 0x11d9fb588>, <built-in method index of list object at 0x11d9fe408>, <built-in method index of list object at 0x11da0c688>, <built-in method index of list object at 0x11da10488>]
[]


In [51]:
df_zoom_fab.tail()

,zsn,catalogName,sellingPrice,brandName,fabricName,tags
361,W0104-005955,Color Kit,490,Multi Brand,14kg Rayon,"[{'attribute': 'Pattern', 'value': 'Plain'}, {..."
362,W0104-005956,Sanvi,1202,Multi Brand,Fancy,"[{'attribute': 'Pattern', 'value': 'Plain'}, {..."
363,W0104-005957,Lize,644,Multi Brand,Royal Silk,"[{'attribute': 'Pattern', 'value': 'Plain'}, {..."
364,W0104-005958,V Desi vol 4,836,Multi Brand,Rayon,"[{'attribute': 'Pattern', 'value': 'Printed'},..."
365,W0104-005960,Shadow,699,Multi Brand,Cotton,"[{'attribute': 'Pattern', 'value': 'Plain'}, {..."


In [46]:
pprint(df_zoom_prach.tags[0]) ;df_zoom_prach.head()

[{'attribute': 'Pattern', 'value': 'Printed'},
 {'attribute': 'Topwear style', 'value': 'Layered'},
 {'attribute': 'Neck', 'value': 'Round'},
 {'attribute': 'Topwear length', 'value': 'Ankle'},
 {'attribute': 'Colour Palette', 'value': 'Dark colours'},
 {'attribute': 'Sleeve style', 'value': 'Sleeveless'}]


,zsn,catalogName,sellingPrice,brandName,fabricName,tags
0,W0102-002553,4 Color Set / D-1082,281,Non Brand,Rayon,"[{'attribute': 'Pattern', 'value': 'Printed'},..."
1,W0102-002551,4 Color Set/ D-1080,234,Non Brand,Rayon,"[{'attribute': 'Pattern', 'value': 'Plain'}, {..."
2,W0102-002552,4 Color Set / D-1341,252,Non Brand,Rayon,"[{'attribute': 'Pattern', 'value': 'Plain'}, {..."
3,W0102-002556,D-1089 / 4 Color Set,252,Non Brand,Rayon,"[{'attribute': 'Pattern', 'value': 'Plain'}, {..."
4,W0102-002550,4 Color Set/D-1081,264,Non Brand,Rayon,"[{'attribute': 'Pattern', 'value': 'Plain'}, {..."


In [45]:
pprint(df_udaan_prach.specList[0]) ;df_udaan_prach.head()

[{'fieldDisplayName': 'Fabric',
  'fieldName': 'ctg_fabric',
  'values': ['Cotton', 'Slub']},
 {'fieldDisplayName': 'Clothing Design / Style',
  'fieldName': 'ctg_design_style',
  'values': ['Straight Kurti', 'Indo-Western Style / Fancy Kurti']},
 {'fieldDisplayName': 'Occasion',
  'fieldName': 'ckr_occasion',
  'values': ['Casual Wear',
             'Party Wear',
             'Summer Wear',
             'Festive Wear',
             'Wedding Wear']},
 {'fieldDisplayName': 'Catalog Type',
  'fieldName': 'ckr_catalog_type',
  'values': ['Color Set']},
 {'fieldDisplayName': 'Size',
  'fieldName': 'size',
  'values': ['42', 'XL', '2XL', '44']},
 {'fieldDisplayName': 'Neck/Collar',
  'fieldName': 'collar',
  'values': ['Round Neck']},
 {'fieldDisplayName': 'Sleeve',
  'fieldName': 'sleeve',
  'values': ['3/4Th Sleeves']},
 {'fieldDisplayName': 'Ideal For',
  'fieldName': 'ctg_ideal_for',
  'values': ['Women']}]


,listingId,title,description,specList,minPrice
0,TLCKR67E2Q0S672DYYFRJ81ZZ47GTCE,sub cotton kurtis 6 pieces set,call whats pap 7096358008,"[{'fieldName': 'ctg_fabric', 'fieldDisplayName...",205
1,TLCKR2XXWJR6DN08514NV4K707TTESL,rayon long designer double layer kurtis 4 colo...,call whats pap 7096358008,"[{'fieldName': 'ctg_ideal_for', 'fieldDisplayN...",NaN
2,TLCKR61D16QYCQCD2945VKBBYYPQ5F9,d no 1217 14kg rayon embroidered kurtis 4 colo...,call whats pap 7096358008,"[{'fieldName': 'ctg_fabric', 'fieldDisplayName...",203.7
3,TLCKR10JHREK9P086LZ748DGT4HS9RK,cotton ghera pattern kurtis 4 color set,call whats pap 7096358008,"[{'fieldName': 'ctg_fabric', 'fieldDisplayName...",165
4,TLCKRBY8TQQECENCT9ZZF8408D2BRET,rayon long designer double layer kurtis 4 colo...,call whats pap 7096358008,"[{'fieldName': 'ctg_ideal_for', 'fieldDisplayN...",240


In [43]:
df_udaan_fab.head()

,listingId,title,description,specList,minPrice
0,TLCKR48P43RQTV084EZJZT8N3LY3N96,fabfirki new leftest gown kurtis size set,fabrics 14kg rayon cotton cora silk embroidery...,"[{'fieldName': 'size', 'fieldDisplayName': 'Si...",NaN
1,TLCKR1MMN8C9B4Z8HRFMJGXDS3E0EDF,fabfirki new leftest designer catalog magi,catalogue mahifabrics heavy 14kg plain rayon p...,"[{'fieldName': 'ctg_ideal_for', 'fieldDisplayN...",NaN
2,TLCKRX2HMX7CEJJCGE4WRE5TPWVQR9C,fabfirki new leftest catalog sanskruti,fashion sheela volume 01fabric sanskrutiavaila...,"[{'fieldName': 'ctg_ideal_for', 'fieldDisplayN...",NaN
3,TLCKR1JTYGW57GC8CEZ4SCKNZNXQKH0,fabfirki rayon print anarkali kurtis catalog b...,cataloguen name blue whale cloth rayon black i...,"[{'fieldName': 'ctg_ideal_for', 'fieldDisplayN...",NaN
4,TLCKRL97CKLS0C58LWZHPMW6ZRPNJYZ,fabfirki rayon fancy embroidery kurtis catalog...,catalog name p a i s l e ya new range of rayon...,"[{'fieldName': 'size', 'fieldDisplayName': 'Si...",NaN


In [31]:
fab = [val.strip(' ') for val in df_zoom_fab['catalogName']]
print(len(fab))
fab.extend([val.strip(' ') for val in df_zoom_prach['catalogName'] if val is not None])
print(len(fab))

366
848


In [32]:
# make personal word list files
base_dir = 'word_lists/'
if not os.path.exists(base_dir):
    os.makedirs(base_dir)
    
with open(base_dir + 'custom_words.txt', 'w') as fp:
    fp.write('\n'.join(fab))

In [33]:
def regex_split_sent(sent: str) -> list:
    """
    Splits a list to contain only words
    """
    
    return re.findall(r"[\w']+", sent)

def correct_misspelt_words_inline(series: pd.Series, enchant_obj) -> pd.Series:
    """
    This function replaces misspelt words in a string which has 
    damerau-levenshtein distance <= 2
    """
    
    for index, sent in enumerate(series):
        new_sent = []
        for word in regex_split_sent(sent.lower()):
            try:
                if enchant_obj.check(word):
                    new_sent.append(word)
                else:
                    try:
                        suggested_word = enchant_obj.suggest(word)[0]
                        new_sent.append(word if damerau_levenshtein_distance(word, suggested_word) > 2 else suggested_word)
                    except IndexError:
                        new_sent.append(word)
                        
            except:
                print(word, " ", enchant_obj.suggest(word), " ", suggested_word)
                
        series[index] = ' '.join(new_sent).lower()



In [34]:
enchant_obj = enchant.DictWithPWL('en_US', 'word_lists/custom_words.txt')
correct_misspelt_words_inline(df_udaan_fab['title'], enchant_obj)
correct_misspelt_words_inline(df_udaan_fab['description'], enchant_obj)
correct_misspelt_words_inline(df_udaan_prach['title'], enchant_obj)
correct_misspelt_words_inline(df_udaan_prach['description'], enchant_obj)

In [35]:
df_udaan_fab.head()

,listingId,title,description,specList,minPrice
0,TLCKR48P43RQTV084EZJZT8N3LY3N96,fabfirki new leftest gown kurtis size set,fabrics 14kg rayon cotton cora silk embroidery...,"[{'fieldName': 'size', 'fieldDisplayName': 'Si...",NaN
1,TLCKR1MMN8C9B4Z8HRFMJGXDS3E0EDF,fabfirki new leftest designer catalog magi,catalogue mahifabrics heavy 14kg plain rayon p...,"[{'fieldName': 'ctg_ideal_for', 'fieldDisplayN...",NaN
2,TLCKRX2HMX7CEJJCGE4WRE5TPWVQR9C,fabfirki new leftest catalog sanskruti,fashion sheela volume 01fabric sanskrutiavaila...,"[{'fieldName': 'ctg_ideal_for', 'fieldDisplayN...",NaN
3,TLCKR1JTYGW57GC8CEZ4SCKNZNXQKH0,fabfirki rayon print anarkali kurtis catalog b...,cataloguen name blue whale cloth rayon black i...,"[{'fieldName': 'ctg_ideal_for', 'fieldDisplayN...",NaN
4,TLCKRL97CKLS0C58LWZHPMW6ZRPNJYZ,fabfirki rayon fancy embroidery kurtis catalog...,catalog name p a i s l e ya new range of rayon...,"[{'fieldName': 'size', 'fieldDisplayName': 'Si...",NaN


In [36]:
df_udaan_prach.head()

,listingId,title,description,specList,minPrice
0,TLCKR67E2Q0S672DYYFRJ81ZZ47GTCE,sub cotton kurtis 6 pieces set,call whats pap 7096358008,"[{'fieldName': 'ctg_fabric', 'fieldDisplayName...",205
1,TLCKR2XXWJR6DN08514NV4K707TTESL,rayon long designer double layer kurtis 4 colo...,call whats pap 7096358008,"[{'fieldName': 'ctg_ideal_for', 'fieldDisplayN...",NaN
2,TLCKR61D16QYCQCD2945VKBBYYPQ5F9,d no 1217 14kg rayon embroidered kurtis 4 colo...,call whats pap 7096358008,"[{'fieldName': 'ctg_fabric', 'fieldDisplayName...",203.7
3,TLCKR10JHREK9P086LZ748DGT4HS9RK,cotton ghera pattern kurtis 4 color set,call whats pap 7096358008,"[{'fieldName': 'ctg_fabric', 'fieldDisplayName...",165
4,TLCKRBY8TQQECENCT9ZZF8408D2BRET,rayon long designer double layer kurtis 4 colo...,call whats pap 7096358008,"[{'fieldName': 'ctg_ideal_for', 'fieldDisplayN...",240
